In [2]:
import psycopg2 as pg
import pandas as pd

connection = pg.connect(host='pgsql-196447.vipserv.org', port=5432, dbname='wbauer_adb', user='wbauer_adb', password='adb2020')


In [3]:
df = pd.read_sql('select * from film',con=connection)
df.columns

Index(['film_id', 'title', 'description', 'release_year', 'language_id',
       'rental_duration', 'rental_rate', 'length', 'replacement_cost',
       'rating', 'last_update', 'special_features', 'fulltext'],
      dtype='object')

## 1. Ile Kategorii filmów mamy w wypożyczalni?

In [4]:
number_of_categories = pd.read_sql('SELECT COUNT(*) FROM category', con=connection)
print(number_of_categories)

   count
0     16


*Odp: Mamy 16 kategorii*





## 2. Wyświetl listę kategorii w kolejności alfabetycznej.

In [5]:
ans = pd.read_sql('SELECT name FROM category ORDER BY name ASC', con=connection)
ans

,name
0,Action
1,Animation
2,Children
3,Classics
4,Comedy
5,Documentary
6,Drama
7,Family
8,Foreign
9,Games


*Odp: w wypożyczalni jest 16 różnych kategorii filmów *

## 3. Znajdź najstarszy i najmłodszy film do wypożyczenia.

In [6]:
ans = pd.read_sql('SELECT MAX(release_year), MIN(release_year) FROM film', con=connection)
ans

,max,min
0,2006,2006


*Odp: Najstarszy i najmłodszy film jest z 2006 roku, czyli de facto nie da się wyróżnić najstarszego i najmłodszego*

## 4. Ile wypożyczeń odbyło się między 2005-07-01 a 2005-08-01?

In [8]:
ans = pd.read_sql("SELECT COUNT(*) FROM rental WHERE rental_date > '2005-07-01 00:00'::timestamp AND rental_date < '2005-08-01 00:00'::timestamp", con=connection)
ans


,count
0,6709


*Odp: jest 6709 takich rekordów*

## 5. Ile wypożyczeń odbyło się między 2010-01-01 a 2011-02-01

In [9]:
ans = pd.read_sql("SELECT COUNT(*) FROM rental WHERE rental_date > '2010-01-01 00:00'::timestamp AND rental_date < '2011-02-01 00:00'::timestamp", con=connection)
ans

,count
0,0


*Odp: Nie ma wypożyczeń które odbyły się pomiędzy 2010-01-01 a 2011-02-01*

## 6. Znajdź największą płatność wypożyczenia.

In [11]:
ans = pd.read_sql('SELECT MAX(amount) \
                   FROM payment \
                   ', con=connection)
ans

,max
0,11.99


*Odp: Największa płatność to 11.99*

## 7. Znajdź wszystkich klientów z Polski, Nigerii lub Bangladeszu.

In [12]:
ans = pd.read_sql("SELECT customer_id, first_name, last_name, country \
                   FROM customer \
                   INNER JOIN address \
                   ON customer.address_id = address.address_id \
                        INNER JOIN city \
                        ON city.city_id = address.city_id \
                            INNER JOIN country \
                            ON country.country_id = city.country_id \
                                WHERE country.country IN ('Poland', 'Nigeria', 'Bangladesh')\
                                ORDER BY country", con=connection)
ans

,customer_id,first_name,last_name,country
0,21,Michelle,Clark,Bangladesh
1,332,Stephen,Qualls,Bangladesh
2,329,Frank,Waggoner,Bangladesh
3,80,Marilyn,Ross,Nigeria
4,198,Elsie,Kelley,Nigeria
5,103,Gladys,Hamilton,Nigeria
6,284,Sonia,Gregory,Nigeria
7,409,Rodney,Moeller,Nigeria
8,286,Velma,Lucas,Nigeria
9,18,Carol,Garcia,Nigeria


## 8. Gdzie mieszkają pracownicy?

In [13]:
ans = pd.read_sql("SELECT city, country\
                   FROM staff \
                   INNER JOIN address \
                   ON staff.address_id = address.address_id \
                        INNER JOIN city \
                        ON city.city_id = address.city_id \
                            INNER JOIN country \
                            ON country.country_id = city.country_id \
                                ORDER BY country", con=connection)
ans

,city,country
0,Woodridge,Australia
1,Lethbridge,Canada


*Odp: pracownicy mieszkają w Woodridge w Australii oraz w Lethbridge w Kanadzie*

## 9. Ilu pracowników mieszka w Argentynie lub Hiszpanii?

*Odp: W Argentynie oraz w Hiszpanii nie mieszka żaden z pracowników*

## 10. Jakie kategorie filmów zostały wypożyczone przez klientów?

In [14]:
ans = pd.read_sql("SELECT DISTINCT category.name\
                   FROM rental \
                   INNER JOIN inventory \
                   ON inventory.inventory_id = rental.inventory_id \
                        INNER JOIN film \
                        ON film.film_id = inventory.film_id \
                            INNER JOIN film_category \
                            ON film.film_id = film_category.film_id \
                                INNER JOIN category \
                                ON category.category_id = film_category.category_id\
                                ORDER BY category.name", con=connection)
ans

,name
0,Action
1,Animation
2,Children
3,Classics
4,Comedy
5,Documentary
6,Drama
7,Family
8,Foreign
9,Games


*Odp: Porównując wynik z wynikiem z zadania 2. można stwierdzić, że wszystkie kategorie filmów zostały wypożyczone przez klientów*

## 11. Znajdź wszystkie kategorie filmów wypożyczonych w Ameryce.

### a) W Ameryce - czyli wypożyczonych w sklepie znajdującym się na kontynencie amerykańckim

In [16]:
ans = pd.read_sql(" SELECT DISTINCT category.name\
                    FROM rental \
                    INNER JOIN inventory \
                        ON inventory.inventory_id = rental.inventory_id \
                    INNER JOIN film \
                        ON film.film_id = inventory.film_id \
                    INNER JOIN film_category \
                        ON film.film_id = film_category.film_id \
                    INNER JOIN category \
                        ON category.category_id = film_category.category_id\
                    WHERE inventory.store_id IN ( \
                        SELECT store_id\
                        FROM store \
                        INNER JOIN address \
                            ON store.address_id = address.address_id \
                        INNER JOIN city \
                            ON city.city_id = address.city_id \
                        INNER JOIN country \
                            ON country.country_id = city.country_id \
                        WHERE country = 'Canada' ) \
                    ORDER BY name", con=connection)
ans

,name
0,Action
1,Animation
2,Children
3,Classics
4,Comedy
5,Documentary
6,Drama
7,Family
8,Foreign
9,Games


### b) W Ameryce - jako państwie zamieszkania klienta

In [22]:
ans = pd.read_sql(" SELECT DISTINCT category.name\
                    FROM rental \
                    INNER JOIN inventory \
                        ON inventory.inventory_id = rental.inventory_id \
                    INNER JOIN film \
                        ON film.film_id = inventory.film_id \
                    INNER JOIN film_category \
                        ON film.film_id = film_category.film_id \
                    INNER JOIN category \
                        ON category.category_id = film_category.category_id\
                    WHERE rental.customer_id IN ( \
                        SELECT customer_id \
                        FROM customer \
                        INNER JOIN address \
                        ON customer.address_id = address.address_id \
                        INNER JOIN city \
                        ON city.city_id = address.city_id \
                            INNER JOIN country \
                            ON country.country_id = city.country_id \
                                WHERE country.country = 'United States' ) \
                    ORDER BY name", con=connection)
ans

,name
0,Action
1,Animation
2,Children
3,Classics
4,Comedy
5,Documentary
6,Drama
7,Family
8,Foreign
9,Games


*Odp: W Amreyce (rozumianej na oba sposoby) wypożyczono wszystkie dostępne kategorie filmów*

## 12. Znajdź wszystkie tytuły filmów, w których grał: Olympia Pfeiffer lub Julia Zellweger lub Ellen Presley

In [18]:
ans = pd.read_sql(" SELECT title, actor.first_name, actor.last_name \
                    FROM film, film_actor, actor \
                    WHERE ((actor.first_name = 'Olympia' AND actor.last_name = 'Pfeiffer') OR\
                    (actor.first_name = 'Julia' AND actor.last_name = 'Zellweger') OR \
                    (actor.first_name = 'Ellen' AND actor.last_name = 'Presley')) AND \
                    film.film_id = film_actor.film_id AND \
                    film_actor.actor_id = actor.actor_id", con=connection)

ans

,title,first_name,last_name
0,Bilko Anonymous,Ellen,Presley
1,Caribbean Liberty,Ellen,Presley
2,Casper Dragonfly,Ellen,Presley
3,Empire Malkovich,Ellen,Presley
4,Floats Garden,Ellen,Presley
...,...,...,...
64,Outlaw Hanky,Julia,Zellweger
65,Panky Submarine,Julia,Zellweger
66,Rider Caddyshack,Julia,Zellweger
67,Won Dares,Julia,Zellweger


# Wnioski:

- W Pythonie, korzystając z dodatkowych pakietów takich jak sqlalchemy lub psycopg2 i pandas, możemy analizować bazy danych korzystając z języka SQL.
- Język SQL bazuje na zapytaniach - tzw. queries, które zaczynają się od instrukcji SELECT, do której można dodać wiele klauzul zmieniających jej działanie.
- Istotnym jest, że w SQL operatorem porównania jest pojedynczy znak '=', a nie jak w języku Python podwójny - '=='.
